### React Agent from Scratch

In this notebook, I’m building a simple ReAct (Reason + Act) agent from scratch.  
The main idea is to let the model think, call tools when needed, and then use the results to give a final answer.

I’ll create:
- a small `ReactAgent` class to talk with the LLM  
- a few basic tools like `find_disease` and `suggest_medicine`  
- and a loop that checks if the model wants to call a tool, runs it, and sends the result back

This is just a basic example to show how the ReAct process works — nothing fancy, just the main idea written in plain Python.


Paper Link : https://arxiv.org/abs/2210.03629

In [10]:
import os
from dotenv import load_dotenv
from groq import Groq


In [5]:
load_dotenv()

True

## A React Prompt

In [151]:
prompt = """
You are a ReAct-style reasoning agent. Your goal is to think step-by-step and perform actions using available tools to help the user.

# Goals
- Understand the user query.
- Reason internally (do not reveal reasoning).
- Call a tool if needed using proper arguments.
- Observe tool results.
- Continue reasoning until you can produce a final answer.
- Return the final answer clearly.
- If the user query is normal, response accordingly

# Tool Usage
You can use the following tools (names are case-sensitive):

1. find_disease
   - Description: Detects the most likely disease from a user's symptom text.
   - Parameters:
       • text (str): User symptom description.

2. suggest_medicine
   - Description: Suggests suitable medicines or home remedies for a detected disease.
   - Parameters:
       • disease (str): The disease name for which medicine suggestions are needed.

# Tool Call Format
When you need to use a tool, respond *only* with a JSON object like this, enclosed in a fenced block:

```tool_call
{"tool": "<TOOL_NAME>", "args": {"param1": "value1", "param2": "value2"}}
"""


In [152]:
class ReactAgent():
    def __init__(self, system=None):
        self.messages = []
        self.client = Groq()
        self.model_name="llama-3.1-8b-instant"
        if system is None:
            self.messages.append({"role": "system", "content": prompt})
        else:
            self.messages.append({"role": "system", "content": system})


    
    def chat(self, request:str):

        self.messages.append({"role":"user", "content":request})
        res = client.chat.completions.create(
        model = self.model_name,
        messages=self.messages
        )
        self.messages.append({"role":"assistant", "content":res.choices[0].message.content})
        return res.choices[0].message.content


## Calling agent - normal usecase

In [216]:
agent = ReactAgent()
agent.chat("HI")

'Hello. How can I assist you today?'

In [217]:
agent.messages

[{'role': 'system',
  'content': '\nYou are a ReAct-style reasoning agent. Your goal is to think step-by-step and perform actions using available tools to help the user.\n\n# Goals\n- Understand the user query.\n- Reason internally (do not reveal reasoning).\n- Call a tool if needed using proper arguments.\n- Observe tool results.\n- Continue reasoning until you can produce a final answer.\n- Return the final answer clearly.\n- If the user query is normal, response accordingly\n\n# Tool Usage\nYou can use the following tools (names are case-sensitive):\n\n1. find_disease\n   - Description: Detects the most likely disease from a user\'s symptom text.\n   - Parameters:\n       • text (str): User symptom description.\n\n2. suggest_medicine\n   - Description: Suggests suitable medicines or home remedies for a detected disease.\n   - Parameters:\n       • disease (str): The disease name for which medicine suggestions are needed.\n\n# Tool Call Format\nWhen you need to use a tool, respond *o

In [218]:
agent.chat("My Name is Badhon")

'Nice to meet you, Badhon. What brings you here today? Do you have any questions or would you like to chat?'

In [219]:
agent.chat("What is my name?")

'Your name is Badhon.'

In [220]:
agent.messages

[{'role': 'system',
  'content': '\nYou are a ReAct-style reasoning agent. Your goal is to think step-by-step and perform actions using available tools to help the user.\n\n# Goals\n- Understand the user query.\n- Reason internally (do not reveal reasoning).\n- Call a tool if needed using proper arguments.\n- Observe tool results.\n- Continue reasoning until you can produce a final answer.\n- Return the final answer clearly.\n- If the user query is normal, response accordingly\n\n# Tool Usage\nYou can use the following tools (names are case-sensitive):\n\n1. find_disease\n   - Description: Detects the most likely disease from a user\'s symptom text.\n   - Parameters:\n       • text (str): User symptom description.\n\n2. suggest_medicine\n   - Description: Suggests suitable medicines or home remedies for a detected disease.\n   - Parameters:\n       • disease (str): The disease name for which medicine suggestions are needed.\n\n# Tool Call Format\nWhen you need to use a tool, respond *o

## React Agent

### A simple tool to check if any diseases can be detected from the user’s text



In [221]:
symptom_keywords = {
            "common_cold": {"runny nose", "stuffy nose", "sneezing", "sore throat", "cough", "mild fever", "fatigue"},
            "influenza": {"high fever", "chills", "body aches", "headache", "dry cough", "fatigue"},
            "strep_throat": {"severe sore throat", "pain swallowing", "swollen tonsils", "white patches", "fever", "no cough"},
            "migraine": {"throbbing headache", "nausea", "light sensitivity", "sound sensitivity", "unilateral headache"},
            "acid_reflux": {"heartburn", "chest burning", "acid taste", "regurgitation", "worse after meals", "worse lying down"},
            "allergic_rhinitis": {"sneezing", "itchy eyes", "watery eyes", "runny nose", "seasonal", "pollen"},
            "covid_19": {"fever", "dry cough", "loss of smell", "loss of taste", "shortness of breath", "fatigue", "sore throat"},
            "gastroenteritis": {"diarrhea", "vomiting", "nausea", "stomach cramps", "low-grade fever"},
        }

In [222]:
def find_disease(text:str)->str:

    cache:dict()={}

    for diseases, symptoms in symptom_keywords.items():
        cache[diseases]=0
        for symptom in symptoms:
            if symptom in text:
                cache[diseases] +=1
    best_match = max(cache, key=cache.get)
    max_count = cache[best_match]

    if max_count == 0:
        return "No matching disease found."

    return best_match


In [223]:
find_disease("I have diarrhea dry cough vomiting")

'gastroenteritis'

### A Tool to Suggest Possible Medicines


In [224]:
medicine_suggestions = {
    "common_cold": ["Paracetamol for fever", "Cough syrup", "Rest and drink plenty of fluids"],
    "influenza": ["Paracetamol or Ibuprofen", "Plenty of rest", "Hydration and warm fluids"],
    "strep_throat": ["Consult a doctor for antibiotics", "Warm salt-water gargle", "Pain relief tablets"],
    "migraine": ["Ibuprofen or Paracetamol", "Rest in a dark room", "Avoid bright light and loud sounds"],
    "acid_reflux": ["Antacids like Gaviscon", "Avoid spicy/fatty foods", "Don’t lie down after meals"],
    "allergic_rhinitis": ["Antihistamines like Cetirizine", "Avoid allergens", "Use nasal spray if needed"],
    "covid_19": ["Paracetamol for fever", "Rest and stay hydrated", "Consult doctor if breathing difficulty"],
    "gastroenteritis": ["Oral rehydration salts (ORS)", "Rest and light diet", "Avoid spicy foods"],
}

In [225]:
def suggest_medicine(disease: str) -> str:
    if disease not in medicine_suggestions:
        return "No medicine information available for this disease."

    meds = medicine_suggestions[disease]
    return f"Suggested medicines/treatments for {disease.replace('_', ' ')}:\n- " + "\n- ".join(meds)


##### Helper

In [226]:
import json
import re

available_tools = {
    "find_disease": find_disease,
    "suggest_medicine": suggest_medicine
}


def parse_tool_call(reply: str):
    pattern = r"```tool_call\s*(\{.*?\})\s*```"
    match = re.search(pattern, reply, re.DOTALL)
    
    if not match:
        return reply

    try:
        data = json.loads(match.group(1))
        tool_name = data.get("tool")
        args = data.get("args", {})

        if tool_name not in available_tools:
            return f"Unknown tool: {tool_name}"

        # Execute tool 
        try:
            result = available_tools[tool_name](**args)
            return f"Tool '{tool_name}' executed successfully.\nResult: {result}"
        except Exception as e:
            return f"Error executing '{tool_name}': {e}"

    except json.JSONDecodeError:
        return "Invalid JSON in tool call."


### Let's test the primitive ReAct agent

In [227]:
agent = ReactAgent()

In [228]:
print(agent.chat("Explain what a ReAct agent is in simple terms."))


A ReAct agent is like a super-smart assistant that helps answer questions and perform tasks by thinking step-by-step. 

Imagine you're using a robot to help you decide what to do in a situation, and it uses tools like a flashlight, a map, and a guidebook to gather information and make decisions.

Here's how a ReAct agent works:

1. You ask the agent a question or give it a task.
2. The agent breaks down the question into smaller parts to understand what you need.
3. It uses special tools (like the flashlight, map, and guidebook) to get the information it needs.
4. The agent looks at the information it has gathered and thinks about the best way to proceed.
5. Finally, the agent gives you the answer or solution to your problem.

So, in simple terms, a ReAct agent is a clever tool that helps you find answers and solve problems by using its own smart thinking and a set of pre-made tools.


In [229]:
res =agent.chat("I have a sore throat, mild fever, and cough. Can you suggest what medicine I can take?")
print(res)


```tool_call
{"tool": "find_disease", "args": {"text": "sore throat, mild fever, and cough"}}```

After detecting the disease, I will use another tool to suggest medications for it.


In [230]:
result = parse_tool_call(res)

In [231]:
result

"Tool 'find_disease' executed successfully.\nResult: common_cold"

In [232]:
res =agent.chat(result)
print(res)


```tool_call
{"tool": "suggest_medicine", "args": {"disease": "common_cold"}}```


In [233]:
result = parse_tool_call(res)
result 

"Tool 'suggest_medicine' executed successfully.\nResult: Suggested medicines/treatments for common cold:\n- Paracetamol for fever\n- Cough syrup\n- Rest and drink plenty of fluids"

In [234]:
res =agent.chat(result)
print(res)


Based on the results, it seems that you have a common cold. To help manage your symptoms, you can consider taking the following:

1. Paracetamol to reduce your fever.
2. A cough syrup to help soothe your cough.
3. Rest as much as possible to help your body recover.
4. Drink plenty of fluids, such as water, tea, or soup, to stay hydrated and help loosen up mucus.

Please note that if your symptoms persist or worsen, it's always best to consult a healthcare professional for further guidance and advice.


## Now, a complete loop to handle queries


In [235]:
agent = ReactAgent()

In [ ]:
MAX_LOOP = 5

def react_loop(query: str, agent: ReactAgent) -> str:
    current_input = query
    print(f"USER QUERY: {query}\n")

    for step in range(1, MAX_LOOP + 1):
        print(f"--Step {step}--")

        # Sendd input to LLM
        llm_res = agent.chat(current_input)
        print(f"\n LLM Response:\n{llm_res}\n")

        # Try to parse and execute any tool call
        tool_output = parse_tool_call(llm_res)
        print(f"Tool Output:\n{tool_output}\n")

        # If LLM result and tool output are same, it means no new action
        if llm_res == tool_output or tool_output is None or "No tool call" in str(tool_output):
            print("Final answer detected or no tool call.")
            return llm_res

        # Otherwise, feed the tool result back as an observation
        observation = f"Observation: {tool_output}"
        print(f"Feeding observation back:\n{observation}\n")
        current_input = observation

    print("Max loop reached; returning last LLM output.")
    return llm_res


In [237]:
print(react_loop("Hi", agent))

USER QUERY: Hi

--Step 1--

 LLM Response:
Hello! It's nice to meet you. Is there anything I can help you with today?

Tool Output:
Hello! It's nice to meet you. Is there anything I can help you with today?

Final answer detected or no tool call.
Hello! It's nice to meet you. Is there anything I can help you with today?


In [238]:
print(react_loop("I have sore throat, mild fever, and cough. What should I take?", agent))

USER QUERY: I have sore throat, mild fever, and cough. What should I take?

--Step 1--

 LLM Response:
```tool_call
{
  "tool": "find_disease", 
  "args": {
    "text": "Sore throat, mild fever, and cough"
  }
}
```
Please wait while I evaluate the tool output.

Tool Output:
Tool 'find_disease' executed successfully.
Result: common_cold

Feeding observation back:
Observation: Tool 'find_disease' executed successfully.
Result: common_cold

--Step 2--

 LLM Response:
Based on the result, it's likely that you have a common cold. I'll use the 'suggest_medicine' tool to provide suggestions for a common cold.

```tool_call
{
  "tool": "suggest_medicine", 
  "args": {
    "disease": "common_cold"
  }
}
```
Please wait while I evaluate the tool output.

Tool Output:
Tool 'suggest_medicine' executed successfully.
Result: Suggested medicines/treatments for common cold:
- Paracetamol for fever
- Cough syrup
- Rest and drink plenty of fluids

Feeding observation back:
Observation: Tool 'suggest_me

### React response

USER QUERY: I have sore throat, mild fever, and cough. What should I take?

--Step 1--

 LLM Response:
```tool_call
{
  "tool": "find_disease", 
  "args": {
    "text": "Sore throat, mild fever, and cough"
  }
}
```
Please wait while I evaluate the tool output.

Tool Output:
Tool 'find_disease' executed successfully.
Result: common_cold

Feeding observation back:
Observation: Tool 'find_disease' executed successfully.
Result: common_cold

--Step 2--

 LLM Response:
Based on the result, it's likely that you have a common cold. I'll use the 'suggest_medicine' tool to provide suggestions for a common cold.

```tool_call
{
  "tool": "suggest_medicine", 
  "args": {
    "disease": "common_cold"
  }
}
```
Please wait while I evaluate the tool output.

Tool Output:
Tool 'suggest_medicine' executed successfully.
Result: Suggested medicines/treatments for common cold:
- Paracetamol for fever
- Cough syrup
- Rest and drink plenty of fluids

Feeding observation back:
Observation: Tool 'suggest_medicine' executed successfully.
Result: Suggested medicines/treatments for common cold:
- Paracetamol for fever
- Cough syrup
- Rest and drink plenty of fluids

--Step 3--

 LLM Response:
Consider taking paracetamol for your fever, using a cough syrup to help with your cough, and resting as much as possible while drinking plenty of fluids to help your body recover from the common cold. If your symptoms worsen or persist, it's always a good idea to consult with a healthcare professional for further guidance.

Tool Output:
Consider taking paracetamol for your fever, using a cough syrup to help with your cough, and resting as much as possible while drinking plenty of fluids to help your body recover from the common cold. If your symptoms worsen or persist, it's always a good idea to consult with a healthcare professional for further guidance.

Final answer detected or no tool call.
Consider taking paracetamol for your fever, using a cough syrup to help with your cough, and resting as much as possible while drinking plenty of fluids to help your body recover from the common cold. If your symptoms worsen or persist, it's always a good idea to consult with a healthcare professional for further guidance.